In [95]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer

# import ensemble methods
from sklearn.ensemble import (
    BaggingClassifier,
    AdaBoostClassifier,
    GradientBoostingClassifier,
    VotingClassifier,
    StackingClassifier,
    RandomForestClassifier
)
from xgboost import XGBClassifier

# import base estimators
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    ConfusionMatrixDisplay,
    RocCurveDisplay,
)
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

import warnings

warnings.filterwarnings(
    "ignore", category=DeprecationWarning
)  # to avoid deprecation warnings


# Feature Exploration, Engineering and Cleaning
Import the data using the following link : "https://full-stack-bigdata-datasets.s3.eu-west-3.amazonaws.com/Machine+Learning+Supervis%C3%A9/stacking/titanic.csv" , and display the first lines. Are there any missing values in the dataset?

In [96]:
print('I\'m fine')

I'm fine


In [97]:
url = "https://full-stack-bigdata-datasets.s3.eu-west-3.amazonaws.com/Machine+Learning+Supervis%C3%A9/stacking/titanic.csv"
dataset = pd.read_csv(url)
print(dataset[0:5])

   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S  


In [98]:
print(f'shape: {dataset.shape}')
print()

print(f'Number of rows: {dataset.shape[0]}')
print(f'Number of columns: {dataset.shape[1]}')
print()

data_desc = dataset.describe(include="all")
print(f'Desc of shape: \n{data_desc}')
print()

shape: (891, 12)

Number of rows: 891
Number of columns: 12

Desc of shape: 
        PassengerId    Survived      Pclass                     Name   Sex  \
count    891.000000  891.000000  891.000000                      891   891   
unique          NaN         NaN         NaN                      891     2   
top             NaN         NaN         NaN  Braund, Mr. Owen Harris  male   
freq            NaN         NaN         NaN                        1   577   
mean     446.000000    0.383838    2.308642                      NaN   NaN   
std      257.353842    0.486592    0.836071                      NaN   NaN   
min        1.000000    0.000000    1.000000                      NaN   NaN   
25%      223.500000    0.000000    2.000000                      NaN   NaN   
50%      446.000000    0.000000    3.000000                      NaN   NaN   
75%      668.500000    1.000000    3.000000                      NaN   NaN   
max      891.000000    1.000000    3.000000                      

In [99]:
# Percentage of missing values for each column
missing_values = (dataset.isnull().sum() / dataset.shape[0]) * 100
print(f'Missed values: \n{missing_values}')

Missed values: 
PassengerId     0.000000
Survived        0.000000
Pclass          0.000000
Name            0.000000
Sex             0.000000
Age            19.865320
SibSp           0.000000
Parch           0.000000
Ticket          0.000000
Fare            0.000000
Cabin          77.104377
Embarked        0.224467
dtype: float64


The following columns have missing values, so we'll have to choose an imputation technique to handle NAs: Age, Cabin, Embarked

What types of variables are present in this dataset? What kind of preprocessing could you run on these variables?

In [100]:
print(f'dataset_type : \n{dataset.dtypes}')

dataset_type : 
PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object


The dataset is a mix of numeric and categorical variables. The quantitative variables have to be normalized, whereas the categorical have to be encoded.

Here are some guidelines you can follow to clean the dataset as well as create new variables (feature engineering).

Create a Name_length variable that measures the number of characters in the variable Name for each observations.

In [101]:
dataset['Name_length'] = dataset['Name'].apply(len)
dataset.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Name_length
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,23
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,51
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,22
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,44
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,24


Create a variable Has_Cabin that indicates whether the passenger has a cabin or not.

In [102]:
dataset['Has_Cabin'] = dataset['Cabin'].notna()

dataset.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Name_length,Has_Cabin
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,23,False
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,51,True
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,22,False
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,44,True
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,24,False


Create a variable FamilySize that gives the size of each passenger's family.

In [103]:
dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1

dataset.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Name_length,Has_Cabin,FamilySize
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,23,False,2
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,51,True,2
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,22,False,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,44,True,2
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,24,False,1


Create a variable IsAlone that indicates whether the passenger is traveling on their own.

In [104]:
dataset['IsAlone'] = dataset['FamilySize'] == 1

dataset.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Name_length,Has_Cabin,FamilySize,IsAlone
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,23,False,2,False
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,51,True,2,False
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,22,False,1,True
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,44,True,2,False
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,24,False,1,True


Extract the title from each passenger in order to create a variable Title

In [105]:
def get_title(name):
    try:
        # Extraction du titre en supposant que le format est "Nom, Titre. Prénom"
        return name.split(", ")[1].split(".")[0]
    except IndexError:
        # Retourne "Unknown" si le format ne correspond pas
        return "Unknown"

dataset['Title'] = dataset['Name'].apply(get_title)

dataset.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Name_length,Has_Cabin,FamilySize,IsAlone,Title
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,23,False,2,False,Mr
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,51,True,2,False,Mrs
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,22,False,1,True,Miss
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,44,True,2,False,Mrs
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,24,False,1,True,Mr


In [106]:
print(dataset['Title'].unique())

['Mr' 'Mrs' 'Miss' 'Master' 'Don' 'Rev' 'Dr' 'Mme' 'Ms' 'Major' 'Lady'
 'Sir' 'Mlle' 'Col' 'Capt' 'the Countess' 'Jonkheer']


In [107]:
# Remplacement des valeurs dans la colonne 'Title' avec un dictionnaire
dataset['Title'] = dataset['Title'].replace({'Mlle': 'Miss', 'Ms': 'Miss', 'Mme': 'Mrs', 'Lady': 'Miss', 'Sir': 'Mr', 'Master':'Mr' })

print(dataset['Title'].unique())

['Mr' 'Mrs' 'Miss' 'Don' 'Rev' 'Dr' 'Major' 'Col' 'Capt' 'the Countess'
 'Jonkheer']


Are any of the remaining titles underrepresented among the observations? If it is the case, group them in a unique modality "Rare"

In [108]:
value_counts = dataset['Title'].value_counts()

value_counts

Title
Mr              558
Miss            186
Mrs             126
Dr                7
Rev               6
Major             2
Col               2
Don               1
Capt              1
the Countess      1
Jonkheer          1
Name: count, dtype: int64

In [109]:
rare_names = ['Don', 'Rev', 'Dr', 'Major', 'Col', 'Capt', 'the Countess', 'Jonkheer']
dataset['Title'] = dataset['Title'].replace(rare_names, 'Rare')

print(dataset['Title'].unique())

['Mr' 'Mrs' 'Miss' 'Rare']


Drop the columns 'PassengerId', 'Name', 'Ticket', 'Cabin', 'SibSp' du dataset. Why don't we need these columns for what's next?

In [110]:
dataset_cleaned = dataset.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin', 'SibSp'])

dataset_cleaned.head()

,Survived,Pclass,Sex,Age,Parch,Fare,Embarked,Name_length,Has_Cabin,FamilySize,IsAlone,Title
0,0,3,male,22.0,0,7.2500,S,23,False,2,False,Mr
1,1,1,female,38.0,0,71.2833,C,51,True,2,False,Mrs
2,1,3,female,26.0,0,7.9250,S,22,False,1,True,Miss
3,1,1,female,35.0,0,53.1000,S,44,True,2,False,Mrs
4,0,3,male,35.0,0,8.0500,S,24,False,1,True,Mr


Separate the features from the target and split the data between train and test (with random_state = 0)

In [111]:
# Separate target variable Y from features X
print("Separating labels from features...")
target_variable = "Survived"

X = dataset_cleaned.drop(columns=[target_variable], axis= 1)
Y = dataset_cleaned[target_variable]

print("...Done.")
print()

print("Y (Target variable):")
print(Y.head())
print("\nX (Features):")
print(X.head())

Separating labels from features...
...Done.

Y (Target variable):
0    0
1    1
2    1
3    1
4    0
Name: Survived, dtype: int64

X (Features):
   Pclass     Sex   Age  Parch     Fare Embarked  Name_length  Has_Cabin  \
0       3    male  22.0      0   7.2500        S           23      False   
1       1  female  38.0      0  71.2833        C           51       True   
2       3  female  26.0      0   7.9250        S           22      False   
3       1  female  35.0      0  53.1000        S           44       True   
4       3    male  35.0      0   8.0500        S           24      False   

   FamilySize  IsAlone Title  
0           2    False    Mr  
1           2    False   Mrs  
2           1     True  Miss  
3           2    False   Mrs  
4           1     True    Mr  


In [112]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0, stratify=Y)

Using the Pipeline and ColumnTransformer, make all the preprocessings at once. Use the KNN imputer to handle the missing values in the numeric variables, and the SimpleImputer for categorical data.

In [113]:
numeric_features = X.select_dtypes(include=["int64", "float64", "bool"]).columns.tolist()
categorical_features = X.select_dtypes(exclude=["int64", "float64", "bool"]).columns.tolist()
print("numeric_features:")
print(numeric_features)
print("\ncategorical_features:")
print(categorical_features)


numeric_features:
['Pclass', 'Age', 'Parch', 'Fare', 'Name_length', 'Has_Cabin', 'FamilySize', 'IsAlone']

categorical_features:
['Sex', 'Embarked', 'Title']


In [114]:
numeric_transformer = Pipeline(steps=[
    ('imputer', KNNImputer()),
    ('scaler', StandardScaler())
])

In [115]:
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(drop="first"))
])

In [116]:
preprocessor = ColumnTransformer(transformers=[
   ('num', numeric_transformer, numeric_features),
   ('cat', categorical_transformer, categorical_features)
])

In [117]:
# Preprocessing on train set
print('performing preprocessing on train set')
print(X_train.head())

X_train = preprocessor.fit_transform(X_train)
print('preprocessor on train set done')
X_train[0:5]
print()

# Preprocessing on test set
print('performing preprocessing on test set')
print(X_test.head())

X_test = preprocessor.transform(X_test)
print('preprocessor on test set done')
X_test[0:5]


performing preprocessing on train set
     Pclass     Sex   Age  Parch     Fare Embarked  Name_length  Has_Cabin  \
502       3  female   NaN      0   7.6292        Q           30      False   
464       3    male   NaN      0   8.0500        S           18      False   
198       3  female   NaN      0   7.7500        Q           32      False   
765       1  female  51.0      0  77.9583        S           36       True   
421       3    male  21.0      0   7.7333        Q           19      False   

     FamilySize  IsAlone Title  
502           1     True  Miss  
464           1     True    Mr  
198           1     True  Miss  
765           2    False   Mrs  
421           1     True    Mr  
preprocessor on train set done

performing preprocessing on test set
     Pclass     Sex   Age  Parch     Fare Embarked  Name_length  Has_Cabin  \
153       3    male  40.5      2  14.5000        S           31      False   
752       3    male  33.0      0   9.5000        S           32      F

array([[ 0.8342464 ,  0.82047558,  2.1035633 , -0.35183846,  0.42980276,
        -0.5383819 ,  0.7186833 , -1.23846995,  1.        ,  0.        ,
         1.        ,  1.        ,  0.        ,  0.        ],
       [ 0.8342464 ,  0.25328956, -0.46765914, -0.44660331,  0.53706514,
        -0.5383819 , -0.56377913,  0.80744793,  1.        ,  0.        ,
         1.        ,  1.        ,  0.        ,  0.        ],
       [ 0.8342464 ,  0.70703837,  5.96039696, -0.03390238,  3.2186246 ,
        -0.5383819 ,  3.28360818, -1.23846995,  0.        ,  0.        ,
         1.        ,  0.        ,  1.        ,  0.        ],
       [ 0.8342464 , -0.12483446, -0.46765914, -0.44660331,  0.32254038,
        -0.5383819 , -0.56377913,  0.80744793,  1.        ,  0.        ,
         1.        ,  1.        ,  0.        ,  0.        ],
       [-1.56084809, -0.42733367, -0.46765914,  0.94944027,  0.32254038,
         1.85741756, -0.56377913,  0.80744793,  0.        ,  0.        ,
         0.        ,  0.  